In [1]:
import ee
import folium

In [2]:
ee.Authenticate()

Enter verification code: 4/1AY0e-g4FgnqNxcYRSot7RNKzE5hrcWpr11ybtr5n2VM2uCY9N8SayfB1LA8

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self

In [5]:
emap = EngineMap()

In [6]:
raster = ee.Image('UMD/hansen/global_forest_change_2015')

In [7]:
emap.add_ee_raster(raster, vis_params={'min': 0, 'max': 100})
emap

In [8]:
params = {
    'bands': ['treecover2000']
}
emap.add_ee_raster(raster, vis_params=params)
emap

In [10]:
params = {
    'bands': ['last_b50', 'last_b40', 'last_b30']
}
emap.add_ee_raster(raster, vis_params=params)
emap

In [11]:
params = {
    'bands': ['loss', 'treecover2000', 'gain']
}
emap.add_ee_raster(raster, params)
emap

In [18]:
params = {
    'bands': ['loss', 'treecover2000', 'gain'],
    'max': [1, 255, 1]
}
emap.add_ee_raster(raster, params)
emap

In [13]:
params = {
    'bands': ['treecover2000'],
    'palette': ['000000', '00FF00']
}
emap.add_ee_raster(raster, params)
emap

In [14]:
params = {
    'bands': ['treecover2000'],
    'palette': ['000000', '00FF00'],
    'max': 100 
}
emap.add_ee_raster(raster, params)
emap

In [16]:
map1 = raster.visualize(bands='treecover2000', palette=['000000', '00FF00'], max=100)
mask = raster.visualize(bands='treecover2000')
newmap = EngineMap()
newmap.add_ee_raster(image = map1.mask(mask))

In [20]:
cover = raster.visualize(bands = 'treecover2000', max = 100, palette = ['000000', '00FF00'])
loss = raster.visualize(bands = 'loss',  max = 100, palette = ['FF0000'])
gain = raster.visualize(bands = 'gain', max = 100, palette = ['0000FF'])

cover_mask = raster.visualize(bands = 'treecover2000')
loss_mask = raster.visualize(bands = 'loss')
gain_mask = raster.visualize(bands = 'gain')

Finalmap = EngineMap()

Finalmap.add_ee_raster(image = cover.mask(cover_mask))
Finalmap.add_ee_raster(image = loss.mask(loss_mask))
Finalmap.add_ee_raster(image = gain.mask(gain_mask))

Finalmap